In [147]:
import tensorflow as tf
import numpy as np
import sys,time,os
from bLSTM import bLSTM


In [172]:
# Understand how to initialize 2 Tensorflow models,
# train them in one session, and save them independently


class model(object):
    
    def __init__(self,size):
        
        self.inp = tf.placeholder(tf.float32,(size,2),'inp')
        self.mat = tf.placeholder(tf.float32,(2,5),'mat')
        
        
        
    def foo(self):
        self.out = tf.matmul(self.inp,self.mat)
        
    
    

In [184]:
small = model(3)
small.foo()
large = model(10)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    saver = tf.train.Saver(max_to_keep=4)

    
    b = sess.run(small.out, feed_dict={small.inp:np.random.random((3,2)),
                                      small.mat:np.random.random((2,5))})
    print(b)
    
    saver.save(sess, save_path + '/Model', global_step=epoch, write_meta_graph=False)

ValueError: No variables to save